In [5]:
import torch

In [6]:
x = torch.rand(3,5)

In [7]:
x

tensor([[0.4320, 0.5769, 0.7935, 0.5890, 0.0791],
        [0.3873, 0.9429, 0.1221, 0.3006, 0.6264],
        [0.1811, 0.0987, 0.3475, 0.6441, 0.7278]])

In [8]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt

In [9]:
train = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [11]:
test = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor())

In [12]:
test

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [14]:
batch_size = 64

train_loader = DataLoader(train, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

In [15]:
for X, y in test_loader:
    print(f'Shape of X[N,C,H,W]: {X.shape}')
    print(f'Shape of y: {y.shape, y.type}')
    break

Shape of X[N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: (torch.Size([64]), torch.int64)


In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [34]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [37]:
def test(dataloader, model, loos_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f'Test Error: \nAccuracy: {(100*correct): >0.1f} %, Avg.loss {test_loss: >8f}')

In [38]:
epochs = 5

for t in range(epochs):
    print(f'Epochs {t+1}\n ---------------')
    train(train_loader,model, loss_fn, optimizer)
    test(test_loader,model, loss_fn)
    print('Done')

Epochs 1
 ---------------
loss: 2.294840 [    0/60000]
loss: 2.290601 [ 6400/60000]
loss: 2.264989 [12800/60000]
loss: 2.264198 [19200/60000]
loss: 2.240744 [25600/60000]
loss: 2.200069 [32000/60000]
loss: 2.218206 [38400/60000]
loss: 2.171299 [44800/60000]
loss: 2.175601 [51200/60000]
loss: 2.139557 [57600/60000]
Test Error: 
Accuracy: 45.5 %, Avg.loss 2.135807
Done
Epochs 2
 ---------------
loss: 2.148025 [    0/60000]
loss: 2.140689 [ 6400/60000]
loss: 2.071841 [12800/60000]
loss: 2.091263 [19200/60000]
loss: 2.034857 [25600/60000]
loss: 1.965549 [32000/60000]
loss: 2.000411 [38400/60000]
loss: 1.908398 [44800/60000]
loss: 1.915700 [51200/60000]
loss: 1.830350 [57600/60000]
Test Error: 
Accuracy: 56.6 %, Avg.loss 1.838402
Done
Epochs 3
 ---------------
loss: 1.878605 [    0/60000]
loss: 1.845754 [ 6400/60000]
loss: 1.718034 [12800/60000]
loss: 1.760356 [19200/60000]
loss: 1.651418 [25600/60000]
loss: 1.602439 [32000/60000]
loss: 1.634363 [38400/60000]
loss: 1.532113 [44800/60000]
lo